# *Telecom X - Análisis de Evasión de Clientes*

*Has sido contratado como asistente de análisis de datos en Telecom X y formarás parte del proyecto "Churn de Clientes". La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.*

*Tu desafío será recopilar, procesar y analizar los datos, utilizando Python y sus principales bibliotecas para extraer información valiosa. A partir de tu análisis, el equipo de Data Science podrá avanzar en modelos predictivos y desarrollar estrategias para reducir la evasión.*

## Instalación de librerias

In [29]:
!!! pip install pandas
!!! pip install matplotlib
!!! pip install numpy

["'!' is not recognized as an internal or external command,",
 'operable program or batch file.']

## Importar librerias

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Importar el dataset

In [31]:
datos = pd.read_json('json/TelecomX_Data.json')

## Visualizar el dataset

In [32]:
datos.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


# Conoce el conjunto de datos

Ahora que has extraído los datos, es fundamental comprender la estructura del dataset y el significado de sus columnas. Esta etapa te ayudará a identificar qué variables son más relevantes para el análisis de evasión de clientes.

In [33]:
datos.info()

<class 'pandas.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   str   
 1   Churn       7267 non-null   str   
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(4), str(2)
memory usage: 340.8+ KB


In [34]:
datos.dtypes

customerID       str
Churn            str
customer      object
phone         object
internet      object
account       object
dtype: object

## Normalizar los diccionarios del JSON

In [35]:
customer_df = pd.json_normalize(datos['customer'])
phone_df = pd.json_normalize(datos['phone'])
internet_df = pd.json_normalize(datos['internet'])
account_df = pd.json_normalize(datos['account'])

## Unir todo en un solo DataFrame

In [36]:
df_flat = pd.concat(
    [
        datos[['customerID', 'Churn']],
        customer_df,
        phone_df,
        internet_df,
        account_df
    ],
    axis=1
)

In [37]:
df_flat.info()

<class 'pandas.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   str    
 1   Churn             7267 non-null   str    
 2   gender            7267 non-null   str    
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   str    
 5   Dependents        7267 non-null   str    
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   str    
 8   MultipleLines     7267 non-null   str    
 9   InternetService   7267 non-null   str    
 10  OnlineSecurity    7267 non-null   str    
 11  OnlineBackup      7267 non-null   str    
 12  DeviceProtection  7267 non-null   str    
 13  TechSupport       7267 non-null   str    
 14  StreamingTV       7267 non-null   str    
 15  StreamingMovies   7267 non-null   str    
 16  Contract          7267 non-null   str    
 17  Paperl

In [38]:
df_flat.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [39]:
use_columns = ['Churn', 'tenure','PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'Charges.Monthly', 'Charges.Total', 'PaymentMethod']

In [40]:
df_use = df_flat[use_columns]
df_use.head()

,Churn,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Charges.Monthly,Charges.Total,PaymentMethod
0,No,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,65.6,593.3,Mailed check
1,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,59.9,542.4,Mailed check
2,Yes,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,73.9,280.85,Electronic check
3,Yes,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,98.0,1237.85,Electronic check
4,Yes,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,83.9,267.4,Mailed check


In [41]:
df_use.info()

<class 'pandas.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7267 non-null   str    
 1   tenure            7267 non-null   int64  
 2   PhoneService      7267 non-null   str    
 3   MultipleLines     7267 non-null   str    
 4   InternetService   7267 non-null   str    
 5   OnlineSecurity    7267 non-null   str    
 6   OnlineBackup      7267 non-null   str    
 7   DeviceProtection  7267 non-null   str    
 8   TechSupport       7267 non-null   str    
 9   StreamingTV       7267 non-null   str    
 10  StreamingMovies   7267 non-null   str    
 11  Contract          7267 non-null   str    
 12  Charges.Monthly   7267 non-null   float64
 13  Charges.Total     7267 non-null   str    
 14  PaymentMethod     7267 non-null   str    
dtypes: float64(1), int64(1), str(13)
memory usage: 851.7 KB


In [42]:
df_use.dtypes

Churn                   str
tenure                int64
PhoneService            str
MultipleLines           str
InternetService         str
OnlineSecurity          str
OnlineBackup            str
DeviceProtection        str
TechSupport             str
StreamingTV             str
StreamingMovies         str
Contract                str
Charges.Monthly     float64
Charges.Total           str
PaymentMethod           str
dtype: object

## Tratar NAN's de la columna "Charges.Total" 

In [48]:
df_use['Charges.Total'] = df_use['Charges.Total'].astype(np.float64)

ValueError: could not convert string to float: ' '

# Transformación (T - Transform)